# 콜백 (Callback)

Model을 학습시킬 때 부가적으로 옵션을 넣어서 수행할 수 있도록 도와주는 객체

모델 학습시 사용하는 .fit()함수에 callbacks라는 파라미터로 지정할 수 있음

딥러닝 모델이 과적합되기 시작하면 새로운 데이터에서의 예측 성능을 신뢰하기가 어려워지기에 학습 시 손실이 더이상 감소하지 않으면 학습을 중단하는 방법으로 사용

=> 특정 조건에서 자동으로 실행되는 기능!



In [11]:
import tensorflow as tf
from tensorflow import keras

In [12]:
# keras 내장 datasets에서 mnist 불러오기
mnist = keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [13]:
# data 정규화 0 ~ 1 사이로
x_train = x_train / x_train.max()
x_test = x_test / 255.

In [14]:
# model 생성
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [18]:
# model compile
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 모델 체크포인드 (Model Checkpoint)

epoch별로 model의 가중치를 저장하는 역할

체크포인트 하이퍼파라미터

- filepath : 체크포인트를 저장하는 경로를 지정
- monitor : 저장 시 기준이 되는 지표를 설정
- save_best_only : monitor 기준 가장 높은 epoch만 저장할지 or epoch마다 저장할지 여부 (True, False)
- verbose : epoch마다 저장 여부를 알려주는 로그메시지 출력여부 (1, 0)
- save_weights_only : 가중치만 저장할지 여부 (True, False) => .ckpt파일에서만 사용 가능

Tensorflow가 업그레이드 되면서 .keras확장자가 새로 생겨남
=> save_weights_only를 사용하려면 Tensorflow의 이전 버전을 사용해야 한다.

In [19]:
# 체크포인트 설정
checkPoint = keras.callbacks.ModelCheckpoint(
    filepath='checkPoint.keras', # .ckpt => .keras : 모델의 가중치 체크포인트 저장 파일
    # 이 파일을 통해서 재학습이 가능, 불필요한 정보도 있어서 파일의 크기가 크고 무거움
    save_best_only=True,
    # True : monitor 되고 있는 값 기준으로 가장 뛰어난 epoch 모델이 저장
    # False : epoch마다 모델이 filepath{epoch} 형태로 저장
    monitor='val_loss', # model을 저장할 때 기준이 되는 값을 지정
    # 테스트 데이터셋을 기준으로 loss가 가장 적을 때 저장하려면 'val_loss'
    # 학습 데이터셋을 기준으로 loss가 가장 적을 때 저장하려면 'loss'
    verbose=1 # 1이면 모델 저장시 문구가 뜸, 0이면 문구없이 저장만
    # save_weights_only=True
)


In [20]:
model.fit(x_train, y_train,
          validation_data=(x_test, y_test),
          epochs=10,
          callbacks=[checkPoint])

Epoch 1/10
1868/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8696 - loss: 0.4242
Epoch 1: val_loss improved from inf to 0.11505, saving model to checkPoint.keras
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 14s 6ms/step - accuracy: 0.8699 - loss: 0.4234 - val_accuracy: 0.9659 - val_loss: 0.1151
Epoch 2/10
1865/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9706 - loss: 0.0948
Epoch 2: val_loss did not improve from 0.11505
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 19s 6ms/step - accuracy: 0.9706 - loss: 0.0948 - val_accuracy: 0.9637 - val_loss: 0.1196
Epoch 3/10
1874/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9798 - loss: 0.0627
Epoch 3: val_loss improved from 0.11505 to 0.08123, saving model to checkPoint.keras
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 21s 6ms/step - accuracy: 0.9798 - loss: 0.0627 - val_accuracy: 0.9753 - val_loss: 0.0812
Epoch 4/10
1873/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9847 - loss: 0.0476
Epoch 4: val_loss improved from 0.08123 to 0.07932, saving model to checkPo

In [21]:
# 체크포인트 적용 전 모델
loss, acc = model.evaluate(x_test, y_test)
print('loss : ', loss, 'accuracy : ', acc)

model.load_weights('checkPoint.keras')

loss, acc = model.evaluate(x_test, y_test)
print('loss : ', loss, 'accuracy : ', acc)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9751 - loss: 0.1043
loss :  0.08753328025341034 accuracy :  0.9785000085830688
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9718 - loss: 0.0959
loss :  0.07931698858737946 accuracy :  0.9765999913215637
